In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import numpy as np
import tensorflow as tf
import keras.metrics as metrics
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
base_dir = '/content/drive/Shareddrives/datasci/ton_iot/result/'

In [3]:
X = []
Y = None
with open(base_dir+'ALL.csv','r') as infile:
  for line in infile:
    pred = line.rstrip().split(',')
    if pred[0] == 'LABEL':
      Y = np.array(pred[1:], dtype='int8').transpose()
    else:
      X.append(pred[1:])
X = np.array(X, dtype='int8').transpose()

In [4]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y, 
                                                    test_size=0.3,
                                                    random_state=42)

In [6]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(len(X[0]),)))
model.add(tf.keras.layers.Dense(64, 
                                activation='relu'))
model.add(tf.keras.layers.Dense(64, 
                                activation='relu'))
model.add(tf.keras.layers.Dense(64, 
                                activation='relu'))
model.add(tf.keras.layers.Dense(64, 
                                activation='relu'))
model.add(tf.keras.layers.Dense(1, 
                                activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', 
              optimizer = 'adam', 
              metrics = ['accuracy', metrics.Precision(), metrics.Recall()])

In [7]:
history = model.fit(X_train,
                    Y_train,
                    epochs=10,
                    verbose = 1)

Epoch 1/10
3026/3026 [==============================] - 15s 4ms/step - loss: 0.0165 - accuracy: 0.9954 - precision: 0.9944 - recall: 0.9925
Epoch 2/10
3026/3026 [==============================] - 7s 2ms/step - loss: 0.0123 - accuracy: 0.9967 - precision: 0.9946 - recall: 0.9959
Epoch 3/10
3026/3026 [==============================] - 7s 2ms/step - loss: 0.0124 - accuracy: 0.9967 - precision: 0.9947 - recall: 0.9958
Epoch 4/10
3026/3026 [==============================] - 6s 2ms/step - loss: 0.0122 - accuracy: 0.9966 - precision: 0.9945 - recall: 0.9959
Epoch 5/10
3026/3026 [==============================] - 6s 2ms/step - loss: 0.0120 - accuracy: 0.9967 - precision: 0.9948 - recall: 0.9959
Epoch 6/10
3026/3026 [==============================] - 6s 2ms/step - loss: 0.0119 - accuracy: 0.9968 - precision: 0.9949 - recall: 0.9960
Epoch 7/10
3026/3026 [==============================] - 7s 2ms/step - loss: 0.0118 - accuracy: 0.9968 - precision: 0.9945 - recall: 0.9963
Epoch 8/10
3026/3026 [====

In [8]:
predict = model.predict(X_test, verbose=1)

1297/1297 [==============================] - 2s 1ms/step


In [9]:
tp,tn,fp,fn = 0,0,0,0
predictn = predict.flatten().round().tolist()
len(predictn)
for i in range(len(predictn)):
  if predictn[i]==1 and Y_test[i]==1:
    tp+=1
  elif predictn[i]==0 and Y_test[i]==0:
    tn+=1
  elif predictn[i]==0 and Y_test[i]==1:
    fp+=1
  elif predictn[i]==1 and Y_test[i]==0:
    fn+=1
    
print(tp,tn,fp,fn)
classification_report(Y_test, predictn, output_dict=True)

14334 26989 54 117


{'0': {'f1-score': 0.9968420469445419,
  'precision': 0.9980031801205488,
  'recall': 0.9956836124843208,
  'support': 27106},
 '1': {'f1-score': 0.9940705294913138,
  'precision': 0.9919036744861948,
  'recall': 0.9962468723936614,
  'support': 14388},
 'accuracy': 0.9958789222538198,
 'macro avg': {'f1-score': 0.9954562882179279,
  'precision': 0.9949534273033718,
  'recall': 0.9959652424389911,
  'support': 41494},
 'weighted avg': {'f1-score': 0.995881026239933,
  'precision': 0.9958881830832159,
  'recall': 0.9958789222538198,
  'support': 41494}}

In [10]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                704       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 13,249
Trainable params: 13,249
Non-trainable params: 0
_________________________________________________________________
None
